# Importing Libraries

In [2]:
import jovian
jovian.configure()
jovian.commit()

[jovian] It looks like Jovian is already configured ( check ~/.jovian/credentials.json ).
Do you want to overwrite the existing configuration? [y/N]: 
[jovian] Skipping..


<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "phanindraparashar/models-01" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/phanindraparashar/models-01


'https://jovian.ml/phanindraparashar/models-01'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import plotly.offline as pyo
import plotly.graph_objs as go
import seaborn as sns
from pycaret.classification import *

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


import pycaret
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import statsmodels.formula.api as smf
from pycaret.regression import *

from statsmodels.formula.api import ols

from sklearn.model_selection import cross_val_score
from pandas_profiling import ProfileReport

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px

from stargazer.stargazer import Stargazer
from IPython.core.display import HTML,Latex

# Helper Functions to create models

In [ ]:
def stat_model_eq(log_model,ex,target='Price_Rent_ratio'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                a += i + ' + '
    b = a[:-2]
    return b

def stat_model(log_model,ex=[],target='Price_Rent_ratio',normalize=True):
    b = stat_model_eq(log_model,ex,target)
    eq = target + ' ~ ' + b
    dvn = normalize_model(log_model,ex)
    if normalize:
        model = ols(eq,data=dvn)
        fitted_model = model.fit()
        #print(fitted_model.summary())
        
    else:
        model = ols(eq,data=log_model)
        fitted_model = model.fit()
        #print(fitted_model.summary())
        
    
    I = fitted_model.pvalues.index
    V = []
    for i in range(len(I)):
        v = fitted_model.pvalues[i]
        if v > .05:
            V.append(I[i])
            #print(I[i],v)
    
    return model
    
    
def normalize_model(df,ex):
    dfw = df.copy()
    p = ['obj_buildingType']
    for i in ex:
        p.append(i)
    var = [c for c in dfw.columns if c not in p]
    dv = dfw[var]
    dvn = (dv - dv.min())/dv.std()
    dvn['obj_buildingType'] = dfw['obj_buildingType']
    return dvn

def eq(di,ex=[],target='Price_Rent_ratio'):
    b = stat_model_eq(di,[],target)
    return target + ' ~ ' + b


def stat_model_eq_interaction_all(log_model,ex=[],target='Price_Rent_ratio'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                if i == 'obj_buildingType':
                    a += i + ' + '
                else:
                    a += 'C(obj_buildingType)*' + i + ' + '
    b = a[:-2]
    return b

def eq_all(di,ex=[],target='Price_Rent_ratio'):
    b = stat_model_eq_interaction_all(di)
    return target + ' ~ ' + b

# Machine Learning Aproach - Not used in the final model

In [ ]:
lin_model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

def lr_pred(dg,target):
    df = dg.copy()

    nc = [c for c in df.columns if c not in ['obj_buildingType']]
    dn = df[nc]
    
    typ_h = pd.get_dummies(df['obj_buildingType'],drop_first=True)
    df = pd.concat([dn,typ_h],axis=1)
    
    cols = [c for c in df.columns if c!= target]
    X = df[cols]
    y = df[target].values
    
    model = lin_model.fit(X, y)
    
    y_pred = model.predict(X)
    
    mse = mean_squared_error(y, y_pred)
    
    r2 = r2_score(y,y_pred)
    
    dr = df.copy()
    dr['pred'] = y_pred
    
    a = dr.copy()
    
    a['diff_pred'] = a['Price_Rent_ratio'] - a['pred']
    a['percemt_diff'] = 100*(a['diff_pred']/a['Price_Rent_ratio'])
    
    dcat = dg.copy()
    dcat['pred'] = y_pred
    dcat['diff_pred'] = dcat['Price_Rent_ratio'] - dcat['pred']
    dcat['percent_diff'] = 100*(dcat['diff_pred']/dcat['Price_Rent_ratio'])
    
    
    return dcat

# Importing the dataset

In [ ]:
dc = pd.read_csv('all_median_pr_merged_with_no_percent_for_elasticity.csv')
mc = dc.drop('zip',axis=1)

# Feature Engineering and Feature creation

In [ ]:
mc['Employed_ratio'] = mc['Gross_domestic_product_per_inhabitant']/mc['Gross_domestic_product_per_person_in_employment']
mc['tax_payers_ratio'] = mc['Wage_and_income_taxpayers']/(mc['Deutsche'] + mc['Foreigners'])
mc['tax_payers_to_employed_ratio'] = mc['tax_payers_ratio']/mc['Employed_ratio']
mc['GDP_dist_state_ratio'] =  mc['Gross_domestic_product_per_inhabitant']/mc['GDP_Per_Capita_state']
mc['tax_per_payers'] = mc['Wage_and_income_tax']/mc['Wage_and_income_taxpayers']

mc['percent_population_18_30'] = mc['population_18_30']/(mc['Deutsche'] + mc['Foreigners'])
mc['percent_population_30_50'] = mc['population_30_50']/(mc['Deutsche'] + mc['Foreigners'])
#mc['German_percentage_of_population'] = 100*mc['Deutsche']/(mc['Deutsche'] + mc['Foreigners'])
mc['foregin_percent_population'] = 100*mc['Foreigners']/(mc['Foreigners']+mc['Deutsche'])

mc.drop(['bus','Total_amount_of_income','Wage_and_income_taxpayers','Gross_domestic_product_per_person_in_employment'],axis=1,inplace=True)
di = mc.drop(['population_18_30','population_30_50','Wage_and_income_tax','Gross_domestic_product_per_inhabitant','Foreigners','Deutsche','avg_space_per_room','price_sup','rent_supply','sell_buy_ratio'],axis=1)

# Removing category of houses that have less than 20 data points to prevent overfitting

In [ ]:
a = lr_pred(di,'Price_Rent_ratio')
a['zip_code'] = a.index
a['abs_percent'] = np.absolute(a['percent_diff'])
count = a.groupby(['obj_buildingType']).count()[['Price_Rent_ratio']]
ag = a.groupby(['obj_buildingType']).mean()[['Price_Rent_ratio','pred','diff_pred','percent_diff','abs_percent','age','rooms']]
ag['count'] = count
ag[ag['count'] > 20]
btype = ag[ag['count'] > 20].index
df = di[di['obj_buildingType'].isin(btype)]
df['area_per_room'] = df['livingSpace']/df['rooms']

df.drop('rooms',axis=1,inplace=True)

# Machine Learning Aproach - Verify if the model is over fitting or not and comparing over 20 ML models

In [ ]:
from pycaret.regression import *
exp_reg = setup(df, target = 'Price_Rent_ratio')

compare_models()

# Linear Model - Machine Learning - (10 cross validation)

In [ ]:
create_model('lr')

# Normalizing the data for intrepretable coefficients

In [ ]:
dsa = dc.copy()
dsa[['zip','obj_buildingType','Price_Rent_ratio','livingSpace','rooms','age','Lat','Lng','Deutsche','Foreigners','dist_aprox','bus', 'cafe', 'doctors', 'fast_food']].head(10)
dvn = normalize_model(df,ex=[])
equ = eq(df)

# Null Model

In [ ]:
model = ols('Price_Rent_ratio ~ 1',data=dvn)
null_model = model.fit()
null_model.summary()

# Helper Functions

In [ ]:
from statsmodels.stats.anova import anova_lm
def mcf(m1,null=null_model):
    a = (m1.llf - m1.df_model)/null.llf
    return 1 - a

def horowitz(m1,m2,null=null_model):
    ro1 = mcf(m1,null)
    ro2 = mcf(m2,null)
    
    print('mcf ro model 1: ' ,ro1)
    print('mcf ro model 2: ',ro2)
    
    
    
    if ro1 > ro2:
        rho = ro1
        rlo = ro2
        print('model 1 has higher mcf')
        deltadf = m1.df_model - m2.df_model
        
        v = -2*(rho-rlo)*null.llf + deltadf
        print(deltadf)
        return -np.sqrt(v)
    else:
        rho = ro2
        rlo = ro1
        print('model 2 has higher mcf')
        deltadf = m2.df_model - m1.df_model
        
        v = -2*(rho-rlo)*null.llf + deltadf
        return -np.sqrt(v)

def f_test(m1,m2):
    num = (m1.ssr - m2.ssr)/(m2.df_model - m1.df_model)
    den = m2.ssr/(m2.nobs - m2.df_model)
    d1 = (m2.df_model - m1.df_model)
    d2 = (m2.nobs - m2.df_model)
    return num/den,d1,d2

def stat_model_eq_interaction_all(log_model,ex=[],target='Price_Rent_ratio'):
    p = [target]
    for j in ex:
        p.append(j)
    a = ''
    V = []
    cols = log_model.columns
    for i in cols:
        if i not in p:
            if i not in V:
                if i == 'obj_buildingType':
                    a += i + ' + '
                else:
                    a += 'C(obj_buildingType)*' + i + ' + '
    b = a[:-2]
    return b

def eq_all(di,ex=[],target='Price_Rent_ratio'):
    b = stat_model_eq_interaction_all(di)
    return target + ' ~ ' + b

# Model with All Variables

In [ ]:
model = ols(eq(df),data=dvn)
basic_model = model.fit()
model.fit().summary()

# all interaction model

In [ ]:
model = ols(eq_all(df),data=dvn)
interaction_all_building = model.fit()
interaction_all_building.summary()

# Helper functions to select relevant interactions using ANOVA

In [ ]:
def drop_interaction(df,interaction_all_building,target='Price_Rent_ratio',dvn=dvn):
    col_interact = [c for c in df.columns if c not in [target,'obj_buildingType']]
    rej = []
    for i in col_interact:
        e = ''
        for j in col_interact:
            if i==j:
                e += j + ' + '
            else:
                e += 'C(obj_buildingType)*' + j + ' + '
        a = e[:-2]
        b = target + ' ~ C(obj_buildingType) + ' + a
        model_i = ols(b,data=dvn)
        imodel = model_i.fit()
        p = anova_lm(imodel,interaction_all_building)['Pr(>F)'].iloc[1]
        if p > .05:
            print('reject the interaction with ',i)
            rej.append(i)
    
    rej.append('obj_buildingType')
    ne = ''
    for i in col_interact:
        if i not in rej:
            ne += 'C(obj_buildingType)*' + i + ' + '
        else:
            ne += i + ' + '
    m = target + ' ~ C(obj_buildingType) + ' + ne[:-2]
    return m,rej

# Selection of interactions

In [ ]:
m,rej = drop_interaction(df,interaction_all_building,target='Price_Rent_ratio')

# Final Model Selected

In [ ]:
model = ols(m,data=dvn)
interaction_selected = model.fit()
interaction_selected.summary()

# Age Quadratic model

In [ ]:
model = ols('Price_Rent_ratio ~ C(obj_buildingType) + C(obj_buildingType)*GDP_Per_Capita_state + Lat + C(obj_buildingType)*Lng + Population_State + cafe + doctors + fast_food + gross_domestic_product + hospital + public_transport + restaurant + zip_population + age + np.power(age,2) + livingSpace + population_density_state + North_of_city + East_of_city + dist_aprox + Employed_ratio + tax_payers_ratio + tax_payers_to_employed_ratio + GDP_dist_state_ratio + tax_per_payers + percent_population_18_30 + percent_population_30_50 + foregin_percent_population + C(obj_buildingType)*area_per_room ',data=df)
interaction_selected_age_quad = model.fit()
interaction_selected_age_quad.summary()

# Linear Model with Age is significantly better - ANOVA TEST

In [ ]:
anova_lm(interaction_selected_age_quad,interaction_selected)

# Significant Final Model

In [ ]:
model = ols('Price_Rent_ratio ~ C(obj_buildingType) + C(obj_buildingType)*GDP_Per_Capita_state + Lat + C(obj_buildingType)*Lng + Population_State + cafe  + fast_food  + public_transport  + C(obj_buildingType)*age + livingSpace + population_density_state  + East_of_city + dist_aprox  + tax_per_payers + percent_population_18_30 + percent_population_30_50  + C(obj_buildingType)*area_per_room ',data=dvn)
interaction_selected_final = model.fit()
interaction_selected_final.summary()

In [ ]:
anova_lm(interaction_selected_final,interaction_selected)

# Results interpretation

In [ ]:
interaction_selected_final.summary()

In [ ]:
report = pd.DataFrame()
models_report = [null_model,basic_model,interaction_all_building,interaction_selected_age_quad,interaction_selected_final]
R2 = []
AR2 = []
AIC = []
DF = []

for i in models_report:
    R2.append(i.rsquared)
    AR2.append(i.rsquared_adj)
    AIC.append(i.aic)
    DF.append(i.df_model)

Models = ['Null Model - non log','Basic Model - non log','All Interactions with building - non Log','Interaction Selected quadratic age - non log','Interaction selected - non log']
report['Models'] = Models
report['R2'] = R2
report['Adj R2'] = AR2
report['AIC'] = AIC
report['DF'] = DF

report_log = pd.read_csv('log_model_report.csv')

In [ ]:
Val = []
Ind = []

for i in range(len(interaction_selected_final.pvalues)):
    if interaction_selected_final.pvalues[i] < .1:
        Ind.append(interaction_selected_final.pvalues.index[i])
        Val.append(interaction_selected_final.params[i])
dp = pd.DataFrame()
dp['Parameters'] = Ind
dp['Coef'] = Val
dp.sort_values('Coef',inplace = True)

In [ ]:
import plotly.graph_objects as go

de = dp.copy()
de['st'] = de['Parameters'].apply(lambda x: 1 if 'C(obj_buildingType)' in x else 0)
de = de[de['st'] == 1]

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'livingSpace' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'Lng' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'age' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'rooms' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

de['nst'] = de['Parameters'].apply(lambda x: 1 if 'GDP' not in x else 0)
de = de[de['nst'] == 1]
de.drop('nst',axis=1,inplace=True)

dhousep = dp.copy()
dhousep = dp.copy()
dhousep = dhousep.iloc[1:5]

dhousep['categ'] = dhousep['Parameters'].apply(lambda x: x.split('[T.')[1].split(']')[0])

# Visualization of coefficients - with p values using plotly

In [ ]:
fig = px.bar(dhousep,x=dhousep['Coef'], y=dhousep['categ'],text=dhousep['Coef'].round(2),
            color='p-values',
            orientation='h'
             )
    
fig.update_layout(
    title='Impact on Category of house',
    xaxis=dict(
        title='Normalized Coefficient',
        titlefont_size=16,
        tickfont_size=14,
    )
    )
    
fig.show()
    

# Helper functions to visualize the coefficients with p values

In [ ]:
def plot_coef_cat(dp,cat):
    dr = dp.copy()
    
    if cat == '':
        cat = 'other features'
        dr['status'] = dr['Parameters'].apply(lambda x: 1 if 'C(obj_buildingType)' not in x else 0)
        dr = dr[dr['status'] == 1]
    
    else:
        dr['status'] = dr['Parameters'].apply(lambda x: 0 if cat not in x else 1)
        dr = dr[dr['status'] == 1]
    

    if cat != '':
        k = list(dr['Parameters'])
        if cat not in k:
            lc = 0
        if cat in k:
            lc = dr[dr['Parameters'] == cat]['Coef'].iloc[0]
        dr['new_Coef'] = dr['Coef'].apply(lambda x: x+lc if x!=lc else x)
    
    else:
        dr['new_Coef'] = dr['Coef']
    
    dr['categ'] = dr['Parameters'].apply(lambda x:  x.split('[T.')[1].split(']:')[0] if '[T.' in x else cat) 
    
    
    fig = px.bar(dr,x=dr['new_Coef'], y=dr['Parameters'],text=dr['new_Coef'].round(2),
            color='p-values',
            orientation='h'
             )
    
    fig.update_layout(
    title='Significant Impact on Price to Rent Ratio due to ' + cat,
    xaxis=dict(
        title='Normalized Coefficient',
        titlefont_size=16,
        tickfont_size=14,
    )
    )
    
    fig.show()
    
    for j in list(dr['categ']):
        if j!= cat:
            scatter_cat(df,j,cat)
    

def scatter_cat(dr,cat,par):
    df = dr.copy()
    df = df[df['obj_buildingType'] == cat][['obj_buildingType',par,'Price_Rent_ratio']]
    
    fig = px.scatter(df, x=par, y='Price_Rent_ratio', trendline="ols")
    fig.update_layout(
    title='Scatter Plot - Price to Rent Ratio  vs ' + cat,
    
    )
    fig.show()
    

# Data Frame of relevant results

In [ ]:
dp = pd.DataFrame()
Param = []
Coef = []
P_V = []

for i in range(len(interaction_selected.params.values)):
    param = interaction_selected.params.index[i]
    coef = interaction_selected.params.values[i]
    p_v = interaction_selected.pvalues[i]
    
    if p_v < .1:
        Param.append(param)
        Coef.append(coef)
        P_V.append(p_v)
dp['Parameters'] = Param
dp['Coef'] = Coef
dp['p-values'] = P_V
dp['p-values'] = dp['p-values'].round(3)
dp.sort_values('p-values')

# Entire Visualization report with scatter plot as proof of validation with data

In [ ]:
plot_coef_cat(dp,'')
plot_coef_cat(dp,'area_per_room')
plot_coef_cat(dp,'Lng')
plot_coef_cat(dp,'livingSpace')
plot_coef_cat(dp,'age')
plot_coef_cat(dp,'GDP_Per_Capita_state')